<a href="https://colab.research.google.com/github/annisaeka123/2341720131_ML_2025/blob/main/JS11/TP_JS11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tugas Praktikum**


1. Buatlah model SVM dengan menggunakan data voice.csv dengan ketentuan,

  - Split data dengan rasio 70:30 dan 80:20 untuk setiap model yang akan dibangun.

  - Gunakan model dengan kernel linier.

  - Gunakan model dengan kernel polynomial.

- Gunakan model dengan kernel RBF.

- Tabulasikan performansi setiap split dan kernel berdasarkan metrik akurasi.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [2]:
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# --------------------------
# 1) SVM on voice.csv
# --------------------------

VOICE_CSV_PATH = "/content/drive/MyDrive/Pembelajaran Mesin/JS11/voice.csv"

# Load
if not os.path.exists(VOICE_CSV_PATH):
    raise FileNotFoundError(f"File not found: {VOICE_CSV_PATH}\nUpload 'voice.csv' ke path ini atau ubah variable VOICE_CSV_PATH.")

df = pd.read_csv(VOICE_CSV_PATH)

# Cek kolom & target.
print("Columns:", df.columns.tolist())
display(df.head())

# Asumsi: label/target kolom bernama 'label' atau 'class' atau 'gender' -> cari otomatis
possible_targets = ['label','class','gender','label_name','voice']
target_col = None
for c in possible_targets:
    if c in df.columns:
        target_col = c
        break
# jika tidak ditemukan, gunakan kolom terakhir sebagai target
if target_col is None:
    target_col = df.columns[-1]

print("Using target column:", target_col)

# Siapkan X, y (drop non-numeric columns except target)
X = df.drop(columns=[target_col])
# jika ada kolom non-numeric (filename/index), hapus
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
X = X[numeric_cols]
y = df[target_col].values

print("Feature shape:", X.shape, "Target distribution:\n", pd.Series(y).value_counts())

# standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fungsi bantu untuk train & evaluate beberapa kernel dan split ratios
def evaluate_svms(X, y, test_size, random_state=42):
    kernels = {
        'linear': SVC(kernel='linear'),
        'poly': SVC(kernel='poly', degree=3),
        'rbf': SVC(kernel='rbf')
    }
    results = []
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    for name, model in kernels.items():
        model.fit(Xtr, ytr)
        ypred = model.predict(Xte)
        acc = accuracy_score(yte, ypred)
        results.append({'split': f"{int((1-test_size)*100)}:{int(test_size*100)}", 'kernel': name, 'accuracy': acc})
    return pd.DataFrame(results)

# Evaluate for 70:30 and 80:20
res_70 = evaluate_svms(X_scaled, y, test_size=0.30)
res_80 = evaluate_svms(X_scaled, y, test_size=0.20)
res = pd.concat([res_70, res_80], ignore_index=True)
print("Summary results (accuracy):")
display(res.pivot_table(index='kernel', columns='split', values='accuracy'))

# Tampilkan classification report untuk model terbaik tiap split (opsional)
print("\nDetailed reports (70:30):")
Xtr, Xte, ytr, yte = train_test_split(X_scaled, y, test_size=0.30, random_state=42, stratify=y)
best70 = SVC(kernel='rbf').fit(Xtr, ytr)  # ubah jika ingin yang terbaik secara praktis
ypred70 = best70.predict(Xte)
print("Accuracy:", accuracy_score(yte, ypred70))
print(classification_report(yte, ypred70))

print("\nDetailed reports (80:20):")
Xtr, Xte, ytr, yte = train_test_split(X_scaled, y, test_size=0.20, random_state=42, stratify=y)
best80 = SVC(kernel='rbf').fit(Xtr, ytr)
ypred80 = best80.predict(Xte)
print("Accuracy:", accuracy_score(yte, ypred80))
print(classification_report(yte, ypred80))

Columns: ['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt', 'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx', 'label']


,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


Using target column: label
Feature shape: (3168, 20) Target distribution:
 male      1584
female    1584
Name: count, dtype: int64
Summary results (accuracy):


split,70:30,80:20
kernel,,
linear,0.978970,0.974763
poly,0.958991,0.957413
rbf,0.983176,0.982650



Detailed reports (70:30):
Accuracy: 0.9831756046267087
              precision    recall  f1-score   support

      female       0.99      0.98      0.98       476
        male       0.98      0.99      0.98       475

    accuracy                           0.98       951
   macro avg       0.98      0.98      0.98       951
weighted avg       0.98      0.98      0.98       951


Detailed reports (80:20):
Accuracy: 0.9826498422712934
              precision    recall  f1-score   support

      female       0.99      0.98      0.98       317
        male       0.98      0.99      0.98       317

    accuracy                           0.98       634
   macro avg       0.98      0.98      0.98       634
weighted avg       0.98      0.98      0.98       634



2. Gunakan data pada praktikum 5 untuk membuat model klasifikasi siang dan malam menggunakan SVM dengan kernel RBF menggunakan fitur histrogram. Gunakan rasio 80:20. Anda dapat bereksperimen dengan hyperparameter tunning dari kernel RBF. Catat performansi akurasinya!

In [5]:
# --------------------------
# 2) SVM RBF with histogram features for Day/Night (praktikum 5)
# --------------------------

import cv2
from glob import glob

IMG_DIR = "/content/drive/MyDrive/Pembelajaran Mesin/JS11/images/training"

def extract_histogram_features(image_path, bins=(32, 32, 32)):
    # read image (BGR) -> convert to RGB
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Cannot read image: {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # compute color histogram (3D) and normalize
    hist = cv2.calcHist([img], [0,1,2], None, bins, [0,256,0,256,0,256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# collect dataset
X_img = []
y_img = []
classes = ['day','night']

for cls in classes:
    folder = os.path.join(IMG_DIR, cls)
    if not os.path.isdir(folder):
        print("Warning: expected folder not found:", folder)
        continue

    for img_path in glob(os.path.join(folder, "*")):
        fname = os.path.basename(img_path)

        # skip macOS hidden files
        if fname.startswith("._"):
            continue

        try:
            feat = extract_histogram_features(img_path)
            X_img.append(feat)
            y_img.append(cls)
        except Exception as e:
            print("skip:", img_path, "->", e)

X_img = np.array(X_img)
y_img = np.array(y_img)

print("Images loaded:", X_img.shape, "Labels:", np.unique(y_img, return_counts=True))

# Standardize features
scaler_img = StandardScaler()
X_img_scaled = scaler_img.fit_transform(X_img)

# Split 80:20
Xtr, Xte, ytr, yte = train_test_split(X_img_scaled, y_img, test_size=0.20,
                                      random_state=42, stratify=y_img)

# --------------------------------------------
# Baseline RBF SVM
# --------------------------------------------
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(Xtr, ytr)
y_pred = svm_rbf.predict(Xte)

base_acc = accuracy_score(yte, y_pred)
print("Baseline RBF accuracy (no tuning):", base_acc)
print(classification_report(yte, y_pred))

# --------------------------------------------
# Hyperparameter tuning (GridSearch RBF)
# --------------------------------------------
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
}

grid = GridSearchCV(SVC(kernel='rbf'),
                    param_grid,
                    cv=5,
                    scoring='accuracy',
                    n_jobs=-1)

grid.fit(Xtr, ytr)

print("Best params:", grid.best_params_)

best_model = grid.best_estimator_

y_pred_best = best_model.predict(Xte)
best_acc = accuracy_score(yte, y_pred_best)

print("Tuned RBF accuracy:", best_acc)
print(classification_report(yte, y_pred_best))
print("Confusion matrix:\n", confusion_matrix(yte, y_pred_best))

# --------------------------------------------
# FINAL SUMMARY TABLE (Fix tanpa append)
# res = hasil voice.csv sebelumnya
# --------------------------------------------
summary = res.copy()

# Baris tambahan (tidak pakai append)
summary.loc[len(summary)] = {
    'split': '80:20',
    'kernel': 'rbf_daynight_baseline',
    'accuracy': base_acc
}

summary.loc[len(summary)] = {
    'split': '80:20',
    'kernel': 'rbf_daynight_tuned',
    'accuracy': best_acc
}

print("\nFinal summary:")
display(summary.pivot_table(index='kernel', columns='split', values='accuracy'))


Images loaded: (240, 32768) Labels: (array(['day', 'night'], dtype='<U5'), array([120, 120]))
Baseline RBF accuracy (no tuning): 0.9583333333333334
              precision    recall  f1-score   support

         day       1.00      0.92      0.96        24
       night       0.92      1.00      0.96        24

    accuracy                           0.96        48
   macro avg       0.96      0.96      0.96        48
weighted avg       0.96      0.96      0.96        48

Best params: {'C': 100, 'gamma': 'auto'}
Tuned RBF accuracy: 1.0
              precision    recall  f1-score   support

         day       1.00      1.00      1.00        24
       night       1.00      1.00      1.00        24

    accuracy                           1.00        48
   macro avg       1.00      1.00      1.00        48
weighted avg       1.00      1.00      1.00        48

Confusion matrix:
 [[24  0]
 [ 0 24]]

Final summary:


split,70:30,80:20
kernel,,
linear,0.978970,0.974763
poly,0.958991,0.957413
rbf,0.983176,0.982650
rbf_daynight_baseline,NaN,0.958333
rbf_daynight_tuned,NaN,1.000000
